# Recurrent Neural Networks

In the previous module, we have been using rich semantic representations of text, and a simple linear classifier on top of the embeddings. What this architecture does is to capture aggregated meaning of words in a sentence, but it does not take into account the **order** of words, because aggregation operation on top of embeddings removed this information from the original text. Because these models are unable to model word ordering, they cannot solve more complex or ambiguous tasks such as text generation or question answering.

To capture the meaning of text sequence, we need to use another neural network architecture, which is caller **recurrent neural network**, or RNN. In RNN, we pass our sentence through the network one symbol at a time, and the network produces some **state**, which we then pass to the network again with the next symbol.

![rnn model](../images/rnn_model.png)

Given the input sequence of tokens $X_0,\dots,X_n$, RNN creates a sequence of neural network blocks, and trains this sequence end-to-end using back propagation. Each network block takes a pair $(X_i,S_i)$ as an input, and produces $S_{i+1}$ as a result. Final state $S_n$ or output $X_n$ goes into a linear classifier to produce the result. All network blocks share the same weights, and are trained end-to-end using one backpropagation pass.

Because state vectors $S_0,\dots,S_n$ are passed through the network, it is able to learn the sequential dependencies between words. For example, when the word *not* appears somewhere in the sequnce, it can learn to negate certain elements within the state vector, resulting in negation.  

Let's see how recurrent neural networks can help us classify our news dataset.

In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab_size = load_dataset()

120000lines [00:04, 27616.79lines/s]
120000lines [00:08, 14194.93lines/s]
7600lines [00:00, 14547.67lines/s]


## Simple RNN Classifier

In case of simple RNN, each recurrent unit is a simple linear network, which takes concatenated input vector and state vector, and produce a new state vector. PyTorch represents this unit with `RNNCell` class, and a networks of such cells - as `RNN` layer.

To define an RNN classifier, we will first apply an embedding layer to lower the dimensionality of input vocabulary, and then have RNN layer on top of it: 

In [83]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

In our case, we will use padded data loader, so each batch will have a number of padded sequences of the same length. RNN layer will take the sequence of embedding tensors, and produce two outputs: 
* $x$ is a sequence of RNN cell outputs at each step
* $h$ is a final hidden state for the last element of the sequence

We then apply a fully-connected linear classifier to get the number of class.

> **Note:** RNNs are quite difficult to train, because once the RNN cells are unrolled along the sequence length, the resulting number of layers involved in back propagation is quite large. Thus we need to select small learning rate, and train the network on larger dataset to produce good results. It can take quite a long time, so using GPU is preferred.

In [84]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3034375
6400: acc=0.3796875
9600: acc=0.449375
12800: acc=0.50359375
16000: acc=0.5425
19200: acc=0.57640625
22400: acc=0.6054017857142857
25600: acc=0.6278515625
28800: acc=0.6466319444444445
32000: acc=0.66471875
35200: acc=0.6782954545454546
38400: acc=0.6909375
41600: acc=0.7018990384615384
44800: acc=0.7116517857142857
48000: acc=0.7199166666666666
51200: acc=0.72759765625
54400: acc=0.735202205882353
57600: acc=0.7419791666666666
60800: acc=0.748092105263158
64000: acc=0.753859375
67200: acc=0.7591517857142858
70400: acc=0.7640767045454545
73600: acc=0.7686277173913043
76800: acc=0.7726692708333334
80000: acc=0.7769875
83200: acc=0.7805408653846154
86400: acc=0.7841666666666667
89600: acc=0.7874888392857143
92800: acc=0.790603448275862
96000: acc=0.79353125
99200: acc=0.7963407258064517
102400: acc=0.7987890625
105600: acc=0.80125
108800: acc=0.8039797794117647
112000: acc=0.8063303571428572
115200: acc=0.8084895833333333
118400: acc=0.8106503378378378


(0.03247815348307292, 0.8117333333333333)

## Long Short Term Memory (LSTM)

One of the main problems of classical RNNs is so-called **vanishing gradients** problem. Because RNNs are trained end-to-end in one back-propagation pass, it is having hard times propagating error to the first layers of the network, and thus the network cannot learn relationships between distant tokens. One of the ways to avoid this problem is to introduce **explicit state management** by using so called **gates**. There are two most known architectures of this kind: **Long Short Term Memory** (LSTM) and **Gated Relay Unit** (GRU).

![LSTM Cell](../images/The_LSTM_Cell.svg)

LSTM Network is organized in a manner similar to RNN, but there are two states that are being passed from layer to layer: actual state $c$, and hidden vector $h$. At each unit, hidden vector $h_i$ is concatenated with input $x_i$, and they control what happens to the state $c$ via **gates**. Each gate is a neural network with sigmoid activation (output in the range $[0,1]$), which can be thought of as bitwise mask when multiplied by the state vector. There are the following gates (from left to right on the picture above):
* **forget gate** takes hidden vector and determines, which components of the vector $c$ we need to forget, and which to pass through. 
* **input gate** takes some information from the input and hidden vector, and inserts it into state.
* **output gate** transforms state via some linear layer with $\tanh$ activation, then selects some of its components using hidden vector $h_i$ to produce new state $c_{i+1}$.

Components of the state $c$ can be thought of as some flags that can be switched on and off. For example, when we encounter a name *Alice* in the sequece, we may want to assume that it refers to female character, and raise the flag in the state that we have female noun in the sentence. When we further encounter phrases *and Tom*, we will raise the flag that we have plural noun. Thus by manipulating state we can supposedly keep track of grammatical properties of sentence parts.

While internal structure of LSTM cell may look complex, PyTorch hides this implementation inside `LSTMCell` class, and provides `LSTM` object to represent the whole LSTM layer. Thus, implementation of LSTM classifier will be pretty similar to the simple RNN which we have seen above:

In [80]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        #print(x.size())
        x = self.embedding(x)
        #print(x.size())
        x,(h,c) = self.rnn(x)
        #print(x.size(),h.size(),c.size())
        return self.fc(h[-1])

Now let's train our network. Note that training LSTM is also quite slow, and you may not seem much raise in accuracy in the beginning of training. Also, you may need to play with `lr` learning rate parameter to find the learning rate that results in reasonable training speed, and yet does not cause 

In [89]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.256875
6400: acc=0.25953125
9600: acc=0.2565625
12800: acc=0.25921875
16000: acc=0.2606875
19200: acc=0.27651041666666665
22400: acc=0.298125
25600: acc=0.3193359375
28800: acc=0.33791666666666664
32000: acc=0.3525
35200: acc=0.3682102272727273
38400: acc=0.3826822916666667
41600: acc=0.3998557692307692
44800: acc=0.41964285714285715
48000: acc=0.43966666666666665
51200: acc=0.46169921875
54400: acc=0.48270220588235296
57600: acc=0.5016145833333333
60800: acc=0.5197532894736843
64000: acc=0.53575
67200: acc=0.5507440476190476
70400: acc=0.5643465909090909
73600: acc=0.5772282608695652
76800: acc=0.5891536458333333
80000: acc=0.6009375
83200: acc=0.6115745192307692
86400: acc=0.6214467592592593
89600: acc=0.6308482142857142
92800: acc=0.6395689655172414
96000: acc=0.6476354166666667
99200: acc=0.6549697580645162
102400: acc=0.662197265625
105600: acc=0.6688352272727273
108800: acc=0.6752941176470588
112000: acc=0.6816339285714286
115200: acc=0.6874652777777778
118400: acc=0.

(0.04210443115234375, 0.6955833333333333)

## Packed Sequences

In our example, we had to pad all sequences in the minibatch with zero vectors. While it results in some memory waste, with RNNs it is more critical that additional RNN cells are created for the padded input items, which take part in training, yet do not carry any important input information. It would be much better to train RNN only to the actual sequence size.

To do that, a special format of padded sequence storage is introduced in PyTorch. Suppose we have input padded minibatch which looks like this:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Here 0 represents padded values, and the actual length vector of input sequences is `[5,3,1]`.

In order to effectively train RNN with padded sequence, we want to begin training first group of RNN cells with large minibatch (`[1,6,9]`), but then end processing of third sequence, and continue training with shorted minibatches (`[2,7]`, `[3,8]`), and so on. Thus, packed sequence is represented as one vector - in our case `[1,6,9,2,7,3,8,4,5]`, and length vector (`[5,3,1]`), from which we can easily reconstruct the original padded minibatch.

To produce packed sequence, we can use `torch.nn.utils.rnn.pack_padded_sequence` function. All recurrent layers, including RNN, LSTM and GRU, support packed sequences as input, and produce packed output, which can be decoded using `torch.nn.utils.rnn.pad_packed_sequence`.

To be able to produce packed sequence, we need to pass length vector to the network, and thus we need a different function to prepare minibatches:

In [94]:
def pad_length(b):
    # first, compute max length of a sequence in this minibatch
    len_seq = [len(x[1]) for x in b]
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t[1]),(0,l-len(t[1])),mode='constant',value=0) for t in b]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Actual network would be very similar to `LSTMClassifier` above, but `forward`

In [97]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        #print(x.size(),h.size(),c.size())
        return self.fc(h[-1])

In [99]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001)


/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


3200: acc=0.285625
6400: acc=0.35359375
9600: acc=0.4163541666666667
12800: acc=0.466875
16000: acc=0.513375
19200: acc=0.5499479166666666
22400: acc=0.5829017857142857
25600: acc=0.6090234375
28800: acc=0.6328125
32000: acc=0.65221875
35200: acc=0.6679829545454545
38400: acc=0.68265625
41600: acc=0.6958413461538462
44800: acc=0.70796875
48000: acc=0.717875
51200: acc=0.72654296875
54400: acc=0.7354411764705883
57600: acc=0.7426388888888888
60800: acc=0.7495888157894737
64000: acc=0.75584375
67200: acc=0.7616369047619047
70400: acc=0.7670738636363637
73600: acc=0.7718070652173913
76800: acc=0.7765755208333334
80000: acc=0.780925
83200: acc=0.7847235576923077
86400: acc=0.7886921296296296
89600: acc=0.7923102678571429
92800: acc=0.7954202586206897
96000: acc=0.7985416666666667
99200: acc=0.8016633064516129
102400: acc=0.804658203125
105600: acc=0.8075189393939394
108800: acc=0.8099816176470588
112000: acc=0.8122767857142857
115200: acc=0.8149479166666667
118400: acc=0.8172972972972973


(0.02941259562174479, 0.8183416666666666)

In [10]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)
# build new `Vocab` object from GloVe vocabulary and load embedding vectors 
voc = torchtext.vocab.build_vocab_from_iterator([vocab.itos])
voc.load_vectors(vocab)
# load the dataset using pre-defined vocabulary
train_dataset, test_dataset = torchtext.datasets.text_classification.DATASETS['AG_NEWS'](root='./data', vocab=voc)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

1lines [00:00, 17.30lines/s]
120000lines [00:08, 13428.63lines/s]
7600lines [00:00, 8951.09lines/s] 


3200: acc=0.25125
6400: acc=0.25
9600: acc=0.248125
12800: acc=0.246875
16000: acc=0.24725
19200: acc=0.24713541666666666
22400: acc=0.24607142857142858


(0.42084818014485365, 0.2473608445297505)

In [12]:
net = RNNClassifier(vocab_size,50,32,len(classes))
net.embedding.weight.data = voc.vectors
net = net.to(device)
train_epoch(net,train_loader, lr=10, epoch_size=25000)

3200: acc=0.249375
6400: acc=0.25
9600: acc=0.2504166666666667
12800: acc=0.249609375
16000: acc=0.25
19200: acc=0.24963541666666667
22400: acc=0.24977678571428572


(5.640533779190659, 0.2492802303262956)

## Recurrent Neural Networks (LSTMs and GRU)

Recurrent Neural Networks (RNNs) and their gated cell variants such as Long Short Term Memory Cells (LSTMs) and Gated Recurrent Units (GRUs) provided a mechanism for modeling word ordering by forwarding the context of each previous prediction into the next evaluation step.



This enabled more complex natrual language processing tasks that require sequence to sequence as well as encoder/decoder mechanisms to be more effectively modeled with neural frameworks such as PyTorch such as Text Translation, Image Captioning, and Named Entity Recognition.

The image below showcases some of the neural tasks that RNNs enabled with neural methods.

![RNN paterns](../images/rnn_tasks.gif)

Additional variations of RNNs such as Bidirectional-RNNs which process text in both left to right and right to left and character level RNNs for enhancing underrepresented or out of vocabulary word embeddings led to many state of the art neural NLP breakthroughs.

One cause for sub-optimal performance with standard LSTM encoder-decoder models for sequence to sequence tasks such as Named Entity Recognition and Machine Translation is that they weighted the impact each input vector evenly on each output vector. In reality specific words in the input sequence often have more impact on sequential outputs at different time steps.

## Attention Mechanisms

**Attention Mechanisms** provide a means of weighting the contextual impact of each input vector on each output prediction of the RNN. 

![attention](images/attention.gif)

An example of an attention mechanism applied to the task of neural translation in Microsoft Translator

Attention mechanisms are responsible for much of the current or near current state of the art in Natural language processing. Adding attention however greatly increases the number of model parameters which led to scaling issues with RNNs. A key constraint of scaling RNNS is that the recurrent nature of the models makes it challenging to batch and parelleize training. In an RNN each element of a sequence needs to be processed in sequential order which means it cannot be easily parallelized.

This adoption of attention mechanisms combined with this constraint led to the creation of the now State of the Art Transformer Models that we know and use today from BERT to OpenGPT3.

## Tranformer Models

Instead of forwarding the context of each previous prediction into the next evaluation step Transformer models use positonal encodings and attention to capture the context of a given input with in a provided window size of text. The image below shows how the positional encodings with attention can capture context with in a given window.

![](images/transformer_explination.gif) 


Since each input position is mapped independently to each output position, transformers can parallelize better than RNNs which enables much larger and more expressive language models. Each attention head can be used to learn different relationships between words that improves downstream Natrual Language Processing tasks.

BERT is a very large multi layer transformer network with (12 layers for BERT-base, and 24 for BERT-large). The model is first pre-trained on large corpus of text data (WikiPedia + books) using un-superwised training (predicting masked words in a sentence). During pre-training the model absorbs significant level of language understanding which can then be leveraged with other datasets using fine tuning. This process is called **transfer learning**. 

![picture from http://jalammar.github.io/illustrated-bert/](images/jalammarBERT-language-modeling-masked-lm.png)

There are many variations of Transformer architectures including BERT, DistilBERT. BigBird, OpenGPT3 and more that can be fine tuned. The HuggingFace package provides repository for training many of these architectures with PyTorch. 

![HuggingFace](images/huggingface.jpg)

In the next module we will be using the HuggingFace Library with PyTorch to fine tune a state of the art DistilBert transformer model for question and answering.
